https://wrds-www.wharton.upenn.edu/data-dictionary/contrib_global_factor/global_factor/

In [1]:
import pandas as pd
import wrds

In [2]:
conn = wrds.Connection()

Loading library list...
Done


In [3]:
begdate = '2015-01-01'
enddate = '2022-12-30'

Read factor data for sp500

In [4]:
sp500 = pd.read_parquet('../data/price_vol_gics_sp500.parquet.gzip')[['permno']].drop_duplicates()
permno_list_sp500 = sp500['permno'].astype('int').to_list()
formatted_permnos_sp500 = ",".join(["'{}'".format(permno) for permno in permno_list_sp500])

In [5]:
query = f"""
SELECT 
    DISTINCT permno, date, 
    ret_exc, dolvol, div12m_me, ret_1_0, ret_6_1, ret_12_1, ret_60_12, at_gr1, ebit_sale, ebit_bev, sale_bev, niq_be, niq_su, ni_ar1, beta_60m, 
    resff3_12_1, resff3_6_1,
    ivol_ff3_21d, iskew_ff3_21d,
    dolvol_126d,
    ami_126d,
    betadown_252d,
    rvolhl_21d
FROM 
    contrib_global_factor.global_factor
WHERE
    permno IN ({formatted_permnos_sp500}) AND date BETWEEN '{begdate}' AND '{enddate}'
"""

factor_sp500 = conn.raw_sql(query)

In [6]:
factor_sp500['date'] = pd.to_datetime(factor_sp500['date'])
factor_sp500 = factor_sp500.sort_values(['permno', 'date'])
factor_sp500

,permno,date,ret_exc,dolvol,div12m_me,ret_1_0,ret_6_1,ret_12_1,ret_60_12,at_gr1,...,ni_ar1,beta_60m,resff3_12_1,resff3_6_1,ivol_ff3_21d,iskew_ff3_21d,dolvol_126d,ami_126d,betadown_252d,rvolhl_21d
20752,10104.0,2015-01-30,-0.065835,1.316969e+10,0.011570,-0.065822,0.116816,0.229675,0.660074,0.154156,...,0.313896,1.343866,0.153869,0.324349,0.006978,0.354046,6.038322e+08,0.000015,0.747223,0.012230
39944,10104.0,2015-02-27,0.046094,9.460449e+09,0.011061,0.046073,0.014666,0.083824,0.646005,0.154156,...,0.313896,1.321035,0.095244,0.329564,0.007480,-0.134348,6.217956e+08,0.000015,0.722099,0.007808
15145,10104.0,2015-03-31,-0.015282,1.489170e+10,0.011295,-0.015290,0.151530,0.083875,0.650775,0.153300,...,0.254190,1.342105,-0.038033,0.489417,0.009419,0.840226,6.039523e+08,0.000016,0.810080,0.010544
24814,10104.0,2015-04-30,0.014339,1.213567e+10,0.011213,0.014368,0.108159,0.064960,0.641161,0.153300,...,0.254190,1.343251,0.035773,0.486609,0.009335,0.368307,6.012961e+08,0.000015,0.815837,0.009331
25864,10104.0,2015-05-29,-0.002975,7.921195e+09,0.012100,-0.002980,0.035024,0.050954,0.933389,0.153300,...,0.254190,1.326061,-0.015913,0.126400,0.004227,-0.286122,5.919505e+08,0.000016,0.835822,0.008163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56815,93436.0,2022-08-31,-0.074390,2.018076e+11,0.000000,-0.072489,0.024149,0.211670,9.336049,0.246659,...,1.377033,2.125661,0.082970,-0.194176,0.018736,-1.804556,2.283048e+10,0.000001,1.679996,0.025712
57829,93436.0,2022-09-30,-0.039518,3.439099e+11,0.000000,-0.037589,-0.232712,0.066217,10.367340,0.246659,...,1.377033,2.071230,-0.132521,-0.530150,0.017906,-0.822199,2.197847e+10,0.000001,1.610794,0.027683
40432,93436.0,2022-10-31,-0.144495,3.942324e+11,0.000000,-0.142168,-0.086143,-0.285682,15.800892,0.242393,...,1.134313,1.940529,-0.045980,-0.181902,0.031803,-1.119941,2.093537e+10,0.000001,1.619264,0.032328
16388,93436.0,2022-11-30,-0.147182,3.665080e+11,0.000000,-0.144326,-0.099755,-0.403700,17.532620,0.242393,...,1.134313,1.896425,-0.205443,-0.428101,0.024830,0.778439,1.990059e+10,0.000001,1.694822,0.038296


Read factor data for our universe

In [7]:
all = pd.read_parquet('../data/price_vol_gics_all.parquet.gzip')[['permno']].drop_duplicates()
permno_list_all = all['permno'].astype('int').to_list()
formatted_permnos_all = ",".join(["'{}'".format(permno) for permno in permno_list_all])

In [8]:
query = f"""
SELECT 
    DISTINCT permno, date, 
    ret_exc, dolvol, div12m_me, ret_1_0, ret_6_1, ret_12_1, ret_60_12, at_gr1, ebit_sale, ebit_bev, sale_bev, niq_be, niq_su, ni_ar1, beta_60m, 
    resff3_12_1, resff3_6_1,
    ivol_ff3_21d, iskew_ff3_21d,
    dolvol_126d,
    ami_126d,
    betadown_252d,
    rvolhl_21d
FROM 
    contrib_global_factor.global_factor
WHERE
    permno IN ({formatted_permnos_all}) AND date BETWEEN '{begdate}' AND '{enddate}'
"""

factor_all = conn.raw_sql(query)

In [9]:
factor_all['date'] = pd.to_datetime(factor_all['date'])
factor_all = factor_all.sort_values(['permno', 'date'])
factor_all

,permno,date,ret_exc,dolvol,div12m_me,ret_1_0,ret_6_1,ret_12_1,ret_60_12,at_gr1,...,ni_ar1,beta_60m,resff3_12_1,resff3_6_1,ivol_ff3_21d,iskew_ff3_21d,dolvol_126d,ami_126d,betadown_252d,rvolhl_21d
74912,10026.0,2015-01-30,-0.097926,1.554613e+08,0.013440,-0.097913,0.215487,0.251327,1.190730,0.091553,...,-0.301492,0.658416,0.240855,1.024676,0.025782,-1.652100,5.605594e+06,0.001615,0.755477,0.011927
9752,10026.0,2015-02-27,0.031309,1.145066e+08,0.013033,0.031288,0.042989,0.070713,1.260923,0.091553,...,-0.301492,0.651721,-0.095124,-0.003300,0.005304,-0.579225,5.912018e+06,0.001608,0.689799,0.007551
151163,10026.0,2015-03-31,0.058018,2.032422e+08,0.012732,0.058010,0.085132,0.065109,1.297270,0.091553,...,-0.301492,0.655078,-0.255279,-0.203031,0.010182,0.760019,6.449754e+06,0.001662,0.671962,0.013122
137604,10026.0,2015-04-30,-0.022241,1.210541e+08,0.013020,-0.022212,0.042554,0.155429,1.090496,0.066340,...,0.108591,0.651477,-0.089993,-0.237293,0.011334,0.193457,6.682720e+06,0.001603,0.748817,0.010195
34408,10026.0,2015-05-29,0.033265,1.014937e+08,0.012601,0.033260,-0.000205,0.128921,1.193766,0.066340,...,0.108591,0.649139,-0.089232,-0.328404,0.007466,0.272187,6.808353e+06,0.001645,0.746321,0.008312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84561,93436.0,2022-08-31,-0.074390,2.018076e+11,0.000000,-0.072489,0.024149,0.211670,9.336049,0.246659,...,1.377033,2.125661,0.082970,-0.194176,0.018736,-1.804556,2.283048e+10,0.000001,1.679996,0.025712
172953,93436.0,2022-09-30,-0.039518,3.439099e+11,0.000000,-0.037589,-0.232712,0.066217,10.367340,0.246659,...,1.377033,2.071230,-0.132521,-0.530150,0.017906,-0.822199,2.197847e+10,0.000001,1.610794,0.027683
60515,93436.0,2022-10-31,-0.144495,3.942324e+11,0.000000,-0.142168,-0.086143,-0.285682,15.800892,0.242393,...,1.134313,1.940529,-0.045980,-0.181902,0.031803,-1.119941,2.093537e+10,0.000001,1.619264,0.032328
24464,93436.0,2022-11-30,-0.147182,3.665080e+11,0.000000,-0.144326,-0.099755,-0.403700,17.532620,0.242393,...,1.134313,1.896425,-0.205443,-0.428101,0.024830,0.778439,1.990059e+10,0.000001,1.694822,0.038296


Save to parquet

In [10]:
factor_sp500.to_parquet('../data/factor_sp500.parquet.gzip', compression='gzip')
factor_all.to_parquet('../data/factor_all.parquet.gzip', compression='gzip')

In [11]:
conn.close()